# Robust Scaling 적용한 lightfm model 학습

In [1]:
import os
import pandas as pd
import numpy as np
from time import time
from tqdm.notebook import tqdm
import pickle

from scipy.sparse import coo_matrix, csr_matrix

from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k

from hyperopt import fmin, hp, tpe, Trials

C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


min_max_item_features.csv
min_max_user_features.csv
rating.csv
robust_item_features.csv
robust_user_features.csv
whisky.csv

In [2]:
item_features = pd.read_csv("dataset/robust_item_features.csv", index_col=0, encoding="UTF-8")
user_features = pd.read_csv("dataset/robust_user_features.csv", index_col=0, encoding="UTF-8")
rating = pd.read_csv("dataset/rating.csv", index_col=0, encoding="UTF-8")
whisky = pd.read_csv("dataset/whisky.csv", index_col=0, encoding="UTF-8")

In [3]:
item_features = csr_matrix(item_features)
user_features = csr_matrix(user_features)

In [4]:
print(user_features)

  (0, 0)	4.0
  (0, 2)	30.0
  (0, 3)	-0.1666666666666666
  (0, 5)	-0.6
  (0, 6)	0.1666666666666666
  (0, 7)	-0.1666666666666666
  (0, 8)	0.3333333333333333
  (0, 9)	1.3333333333333333
  (0, 10)	0.8571428571428571
  (0, 11)	1.3333333333333333
  (0, 12)	0.8571428571428571
  (0, 13)	1.3461538461538465
  (1, 0)	5.0
  (1, 1)	0.4
  (1, 3)	-0.1666666666666666
  (1, 4)	-0.3333333333333333
  (1, 5)	0.2
  (1, 6)	0.5
  (1, 7)	0.8333333333333334
  (1, 8)	0.6666666666666666
  (1, 9)	0.6666666666666666
  (1, 10)	0.5714285714285714
  (1, 12)	1.4285714285714286
  (1, 13)	-0.1923076923076923
  (2, 0)	4.0
  :	:
  (119512, 13)	-0.1923076923076923
  (119513, 0)	1.0
  (119513, 1)	0.44
  (119513, 3)	-0.1666666666666666
  (119513, 4)	0.3
  (119513, 5)	-1.0
  (119513, 6)	-0.8333333333333334
  (119513, 7)	-0.5
  (119513, 8)	1.3333333333333333
  (119513, 10)	0.4285714285714285
  (119513, 11)	-0.6666666666666666
  (119513, 12)	-0.2857142857142857
  (119513, 13)	-0.1923076923076923
  (119514, 0)	1.0
  (119514, 1)	

In [5]:
unique_user = rating["user_id"].unique()

In [6]:
unique_user

array([     0,      1,      2, ..., 119512, 119513, 119514], dtype=int64)

### make Interactions 

In [7]:
rating.shape

(908176, 3)

In [8]:
def create_user_item_interactions(rating, n_users, n_items):
    interactions = coo_matrix(
        (
            rating['rating'].values,
            (rating['user_id'].values, rating["whisky_id"].values)
        ),
        shape=(n_users, n_items)
    )
    return interactions

In [9]:
interactions = create_user_item_interactions(rating, rating["user_id"].nunique(), whisky["whisky_id"].nunique())

In [10]:
print(interactions)
interactions

  (0, 0)	9.0
  (1, 0)	7.0
  (2, 0)	9.0
  (3, 0)	10.0
  (4, 0)	9.0
  (5, 0)	7.0
  (6, 0)	10.0
  (7, 0)	7.0
  (8, 0)	10.0
  (9, 0)	10.0
  (10, 0)	9.0
  (11, 0)	9.0
  (12, 0)	8.0
  (13, 0)	8.0
  (14, 0)	10.0
  (15, 0)	10.0
  (16, 0)	9.0
  (17, 0)	9.0
  (18, 0)	8.0
  (19, 0)	9.0
  (20, 0)	9.0
  (21, 0)	10.0
  (22, 0)	9.0
  (23, 0)	9.0
  (24, 0)	9.0
  :	:
  (7636, 3531)	4.0
  (56292, 3531)	2.0
  (2670, 3531)	2.0
  (786, 3531)	2.0
  (5942, 3531)	6.0
  (15561, 3531)	2.0
  (25623, 3532)	6.0
  (2885, 3532)	6.0
  (7368, 3533)	2.0
  (69554, 3534)	10.0
  (4484, 3534)	2.0
  (85727, 3534)	5.0
  (43900, 3534)	4.0
  (16070, 3534)	8.0
  (29998, 3534)	8.0
  (7895, 3534)	4.0
  (104052, 3534)	5.0
  (31152, 3534)	2.0
  (119512, 3534)	6.0
  (2509, 3534)	4.0
  (4828, 3534)	4.0
  (119513, 3534)	2.0
  (119514, 3534)	8.0
  (3123, 3534)	4.0
  (95648, 3534)	2.0


<119515x3535 sparse matrix of type '<class 'numpy.float64'>'
	with 908176 stored elements in COOrdinate format>

## Train_Test data split

In [11]:
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.2, random_state=42)

In [12]:
train_interactions, valid_interactions = random_train_test_split(train_interactions, test_percentage=0.2, random_state=42)

## Optuna 사용한 HyperParameter 최적화
---
- learning_rate, alpha 등의 하이퍼 파라미터 값을 작게 설정했을 때보다 높게 설정했을 때 AUC Score가 높게 나오는 경향이 보였습니다.
- HyperOPT는 베이지안 최적화 접근 기반인데 이 부분에 제대로 알지 못하기 때문에 전체를 돌려보는 Optuna 방식으로 변경하겠습니다.
- Optuna의 경우 시각화도 가능하고, GridSearchCV보다 빠르다는 장점이 있습니다.

In [13]:
import optuna

In [14]:
def objective(trial):
    
    
    # 조정할 하이퍼 파라미터
    params = {
        "learning_schedule": 'adagrad',
        "loss": "warp",
        "random_state": 0,
        "no_components": trial.suggest_categorical("no_components", [10, 20, 30, 40, 50]),
        'learning_rate': trial.suggest_categorical("learning_rate", [0.0005, 0.001, 0.005,  0.01,  0.05]),
        'item_alpha': trial.suggest_categorical("item_alpha", [5e-04, 1e-03, 5e-03, 1e-02]),
        'user_alpha': trial.suggest_categorical("user_alpha", [5e-04, 1e-03, 5e-03, 1e-02]),
    }

    model = LightFM(**params)

    model.fit(interactions=train_interactions,
              user_features=user_features,
              item_features=item_features,
              epochs=3,
              verbose=True)

    test_precision = precision_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
    test_recall = recall_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
    test_auc = auc_score(model, valid_interactions, item_features=item_features, user_features=user_features).mean()
    
    print("no_comp: {}, lrn_rate: {:.5f}, item_alpha: {:.5f}, user_alpha: {:.5f}, precision: {:.5f}, recall: {:.5f}, auc_score: {:.5f}".format(
      params["no_components"], params["learning_rate"], params["item_alpha"], params["user_alpha"], test_precision, test_recall, test_auc))
    return test_auc

In [15]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(study.best_trial.params)

[I 2023-03-13 20:39:12,895] A new study created in memory with name: no-name-a06ad83c-6795-4de4-b42e-5be385d06427
Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [01:06<00:00, 22.09s/it]
[I 2023-03-13 20:42:51,232] Trial 0 finished with value: 0.7774629592895508 and parameters: {'no_components': 20, 'learning_rate': 0.005, 'item_alpha': 0.01, 'user_alpha': 0.0005}. Best is trial 0 with value: 0.7774629592895508.


no_comp: 20, lrn_rate: 0.00500, item_alpha: 0.01000, user_alpha: 0.00050, precision: 0.00288, recall: 0.01245, auc_score: 0.77746


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [02:44<00:00, 54.69s/it]
[I 2023-03-13 20:50:32,939] Trial 1 finished with value: 0.7648791670799255 and parameters: {'no_components': 50, 'learning_rate': 0.0005, 'item_alpha': 0.0005, 'user_alpha': 0.01}. Best is trial 0 with value: 0.7774629592895508.


no_comp: 50, lrn_rate: 0.00050, item_alpha: 0.00050, user_alpha: 0.01000, precision: 0.00165, recall: 0.00594, auc_score: 0.76488


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [01:37<00:00, 32.40s/it]
[I 2023-03-13 20:55:28,678] Trial 2 finished with value: 0.48272180557250977 and parameters: {'no_components': 30, 'learning_rate': 0.05, 'item_alpha': 0.0005, 'user_alpha': 0.001}. Best is trial 0 with value: 0.7774629592895508.


...

no_comp: 40, lrn_rate: 0.00500, item_alpha: 0.00100, user_alpha: 0.00100, precision: 0.00165, recall: 0.00539, auc_score: 0.68790


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [02:43<00:00, 54.52s/it]
[I 2023-03-14 02:01:50,632] Trial 48 finished with value: 0.7817395925521851 and parameters: {'no_components': 50, 'learning_rate': 0.005, 'item_alpha': 0.001, 'user_alpha': 0.001}. Best is trial 36 with value: 0.7817395925521851.


no_comp: 50, lrn_rate: 0.00500, item_alpha: 0.00100, user_alpha: 0.00100, precision: 0.00419, recall: 0.02188, auc_score: 0.78174


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [00:50<00:00, 16.68s/it]
[I 2023-03-14 02:04:32,212] Trial 49 finished with value: 0.7769889235496521 and parameters: {'no_components': 20, 'learning_rate': 0.005, 'item_alpha': 0.0005, 'user_alpha': 0.001}. Best is trial 36 with value: 0.7817395925521851.


no_comp: 20, lrn_rate: 0.00500, item_alpha: 0.00050, user_alpha: 0.00100, precision: 0.00291, recall: 0.01235, auc_score: 0.77699
{'no_components': 50, 'learning_rate': 0.005, 'item_alpha': 0.001, 'user_alpha': 0.001}


In [16]:
print("Best Params : {}".format(study.best_params))
print()
print("Best Trials : {}".format(study.best_trials))
print()
print("Best Values : {}".format(study.best_value))

Best Params : {'no_components': 50, 'learning_rate': 0.005, 'item_alpha': 0.001, 'user_alpha': 0.001}

Best Trials : [FrozenTrial(number=36, state=TrialState.COMPLETE, values=[0.7817395925521851], datetime_start=datetime.datetime(2023, 3, 14, 0, 34, 37, 839), datetime_complete=datetime.datetime(2023, 3, 14, 0, 43, 20, 877070), params={'no_components': 50, 'learning_rate': 0.005, 'item_alpha': 0.001, 'user_alpha': 0.001}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'no_components': CategoricalDistribution(choices=(10, 20, 30, 40, 50)), 'learning_rate': CategoricalDistribution(choices=(0.0005, 0.001, 0.005, 0.01, 0.05)), 'item_alpha': CategoricalDistribution(choices=(0.0005, 0.001, 0.005, 0.01)), 'user_alpha': CategoricalDistribution(choices=(0.0005, 0.001, 0.005, 0.01))}, trial_id=36, value=None), FrozenTrial(number=41, state=TrialState.COMPLETE, values=[0.7817395925521851], datetime_start=datetime.datetime(2023, 3, 14, 1, 6, 6, 913256), datetime_complete=date

In [14]:
m1 = LightFM(loss='warp', learning_schedule="adagrad", random_state=0, no_components=60, learning_rate=0.01, item_alpha=0.005, user_alpha=0.01)
m1.fit(interactions=train_interactions, user_features=user_features, item_features=item_features, epochs=5, verbose=True)

Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [04:13<00:00, 50.73s/it]


In [15]:
test_precision = precision_at_k(m1, test_interactions, user_features=user_features, item_features=item_features, k=9).mean()
test_recall = recall_at_k(m1, test_interactions,  user_features=user_features, item_features=item_features, k=9).mean()
test_auc = auc_score(m1, test_interactions, user_features=user_features, item_features=item_features).mean()

print("Train precision: %.5f" % test_precision)
print("Test precision: %.5f" % test_recall) 
print("Test AUC Score : %.5f" % test_auc)

Train precision: 0.00624
Test precision: 0.03233
Test AUC Score : 0.78300


In [17]:
def objective(trial):
    # 조정할 하이퍼 파라미터
    params = {
        "learning_schedule": 'adagrad',
        "loss": "warp",
        "random_state": 0,
        "no_components": trial.suggest_int("no_components", 40, 100, 10),
        'learning_rate': trial.suggest_float("learning_rate", 0.005,  0.01),
        'item_alpha': trial.suggest_float("item_alpha", 1e-03, 1e-02),
        'user_alpha': trial.suggest_float("user_alpha", 1e-03, 1e-02),
    }

    model = LightFM(**params)

    model.fit(interactions=train_interactions,
              user_features=user_features,
              item_features=item_features,
              epochs=5,
              verbose=True)

    test_precision = precision_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
    test_recall = recall_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
    test_auc = auc_score(model, valid_interactions, item_features=item_features, user_features=user_features).mean()
    
    print("no_comp: {}, lrn_rate: {:.5f}, item_alpha: {:.5f}, user_alpha: {:.5f}, precision: {:.5f}, recall: {:.5f}, auc_score: {:.5f}".format(
      params["no_components"], params["learning_rate"], params["item_alpha"], params["user_alpha"], test_precision, test_recall, test_auc))
    return test_auc

In [18]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(study.best_trial.params)

[I 2023-03-14 09:43:24,995] A new study created in memory with name: no-name-33e1116d-f16f-4c9f-aa0b-10e582584106
Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [03:21<00:00, 40.29s/it]
[I 2023-03-14 09:50:42,438] Trial 0 finished with value: 0.7664042711257935 and parameters: {'no_components': 50, 'learning_rate': 0.008515490987966674, 'item_alpha': 0.009989801052029564, 'user_alpha': 0.006525796566942141}. Best is trial 0 with value: 0.7664042711257935.


no_comp: 50, lrn_rate: 0.00852, item_alpha: 0.00999, user_alpha: 0.00653, precision: 0.00224, recall: 0.00890, auc_score: 0.76640


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [02:44<00:00, 32.81s/it]
[I 2023-03-14 09:56:34,061] Trial 1 finished with value: 0.6648830771446228 and parameters: {'no_components': 40, 'learning_rate': 0.00928604382460443, 'item_alpha': 0.001431535015211974, 'user_alpha': 0.008787731876146091}. Best is trial 0 with value: 0.7664042711257935.


no_comp: 40, lrn_rate: 0.00929, item_alpha: 0.00143, user_alpha: 0.00879, precision: 0.00141, recall: 0.00443, auc_score: 0.66488


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [03:20<00:00, 40.04s/it]
[I 2023-03-14 10:03:37,357] Trial 2 finished with value: 0.7677977681159973 and parameters: {'no_components': 50, 'learning_rate': 0.008668578547255225, 'item_alpha': 0.009478209543015877, 'user_alpha': 0.001899806983471075}. Best is trial 2 with value: 0.7677977681159973.


...

no_comp: 60, lrn_rate: 0.00899, item_alpha: 0.00822, user_alpha: 0.00722, precision: 0.00577, recall: 0.03145, auc_score: 0.78108


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [05:46<00:00, 69.30s/it]
[I 2023-03-14 14:20:10,438] Trial 23 finished with value: 0.6730281710624695 and parameters: {'no_components': 70, 'learning_rate': 0.008889179552784395, 'item_alpha': 0.008340193928602932, 'user_alpha': 0.0072747823590638945}. Best is trial 21 with value: 0.7828404903411865.


no_comp: 70, lrn_rate: 0.00889, item_alpha: 0.00834, user_alpha: 0.00727, precision: 0.00232, recall: 0.01134, auc_score: 0.67303


Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 5/5 [03:21<00:00, 40.37s/it]
[W 2023-03-14 14:24:44,346] Trial 24 failed with parameters: {'no_components': 50, 'learning_rate': 0.008177141786385261, 'item_alpha': 0.008041511437748481, 'user_alpha': 0.007254423593822881} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\SSAFY\AppData\Local\Temp\ipykernel_26048\3567443769.py", line 21, in objective
    test_precision = precision_at_k(model, valid_interactions, k=9, item_features=item_features, user_features=user_features).mean()
  File "C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\lightfm\evaluation.py", line 71, in precision_at_k
    ranks = model.predict_rank(
  File "C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-pa

KeyboardInterrupt: 

In [19]:
print("Best Params : {}".format(study.best_params))
print()
print("Best Trials : {}".format(study.best_trials))
print()
print("Best Values : {}".format(study.best_value))

Best Params : {'no_components': 60, 'learning_rate': 0.008146396234592687, 'item_alpha': 0.00628651909946878, 'user_alpha': 0.005732287458834077}

Best Trials : [FrozenTrial(number=21, state=TrialState.COMPLETE, values=[0.7828404903411865], datetime_start=datetime.datetime(2023, 3, 14, 13, 45, 19, 109162), datetime_complete=datetime.datetime(2023, 3, 14, 13, 57, 23, 590782), params={'no_components': 60, 'learning_rate': 0.008146396234592687, 'item_alpha': 0.00628651909946878, 'user_alpha': 0.005732287458834077}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'no_components': IntDistribution(high=100, log=False, low=40, step=10), 'learning_rate': FloatDistribution(high=0.01, log=False, low=0.005, step=None), 'item_alpha': FloatDistribution(high=0.01, log=False, low=0.001, step=None), 'user_alpha': FloatDistribution(high=0.01, log=False, low=0.001, step=None)}, trial_id=21, value=None)]

Best Values : 0.7828404903411865


no_comp: 60, lrn_rate: 0.00815, item_alpha: 0.00629, user_alpha: 0.00573, precision: 0.00543, recall: 0.02894, auc_score: 0.78284

Best Params : {'no_components': 50, 'learning_rate': 0.005, 'item_alpha': 0.001, 'user_alpha': 0.001}
Best Values : 0.7817395925521851

### Evaluation

In [20]:
m1 = LightFM(
    no_components=60,
    learning_schedule='adagrad',
    loss='warp',
    learning_rate=0.01,
    item_alpha=0.00629,
    user_alpha=0.00573,
    random_state=0
)

In [21]:
%time m1.fit(interactions=train_interactions, user_features=user_features, item_features=item_features, epochs=10, verbose=True)

Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [08:16<00:00, 49.70s/it]

CPU times: total: 8min 16s
Wall time: 8min 16s


In [22]:
test_precision = precision_at_k(m1, test_interactions, user_features=user_features, item_features=item_features, k=9).mean()
test_recall = recall_at_k(m1, test_interactions,  user_features=user_features, item_features=item_features, k=9).mean()
test_auc = auc_score(m1, test_interactions, user_features=user_features, item_features=item_features).mean()

print("Train precision: %.5f" % test_precision)
print("Test precision: %.5f" % test_recall) 
print("Test AUC Score : %.5f" % test_auc)

Train precision: 0.00652
Test precision: 0.03355
Test AUC Score : 0.79624


### Result
---
Min-Max Normalization이 조금 더 좋은 성능을 보이기 때문에 Min-Max Normalization을 적용한 model을 저장합니다.

### save

In [ ]:
# # save the model to a file
# with open('robust_lightfm.pkl', 'wb') as f:
#     pickle.dump(model, f)